In [3]:
import networkx as nx

import pandas as pd
import numpy as np

import pickle

For a first run through, I'm going to create a network of ONLY just double verified. Once I get the code down, I'll do others

In [4]:
with open('/Users/robertpagano/metis_data/project_5/final_double_verified_df.pickle', 'rb') as f:
    df_double_verified = pickle.load(f)

In [5]:
df_double_verified.head()

tweet_id   user_id         display_name  screen_name  \
3494  1105324545257426944  15787371       Alison Klayman      aliklay   
8     1105323825296629760  28049003       William Gibson  GreatDismal   
38    1105323801468780544  28049003       William Gibson  GreatDismal   
1015  1105322632956633088  23314049  yvette nicole brown          YNB   
1136  1105322497703137281    784304                 SXSW         sxsw   

                          tweet_date  \
3494  Tue Mar 12 04:28:02 +0000 2019   
8     Tue Mar 12 04:25:10 +0000 2019   
38    Tue Mar 12 04:25:04 +0000 2019   
1015  Tue Mar 12 04:20:26 +0000 2019   
1136  Tue Mar 12 04:19:54 +0000 2019   

                                              main_text  fav_count  rt_count  \
3494  RT @schwarz: Luke Thompson, the guy who kicked...          0       623   
8     RT @ddale8: Ocasio-Cortez has offered this sem...          0        80   
38    RT @ddale8: This is not true. Right to Rise, t...          0       212   
1015  RT @VanityFair: NEW: @AOC opens up about the p...          0       371   
1136  RT @sxsw: Watch Representative Alexandria Ocas...          0       367   

      followers_count  is_verified    ...     retweet_fav_count  \
3494             4857         True    ...                1897.0   
8              247650         True    ...                 347.0   
38             247650         True    ...                 718.0   
1015           389468         True    ...                2403.0   
1136           656419         True    ...                1625.0   

      retweet_followers_count quoted_text quoted_id  quoted_user_id  \
3494                  30242.0         NaN       NaN             NaN   
8                    542874.0         NaN       NaN             NaN   
38                   542874.0         NaN       NaN             NaN   
1015                4855698.0         NaN       NaN             NaN   
1136                 656419.0         NaN       NaN             NaN   

      quoted_screen_name quoted_is_verified quoted_fav_count  is_retweet  \
3494                 NaN                NaN              NaN           1   
8                    NaN                NaN              NaN           1   
38                   NaN                NaN              NaN           1   
1015                 NaN                NaN              NaN           1   
1136                 NaN                NaN              NaN           1   

      is_quote  
3494         0  
8            0  
38           0  
1015         0  
1136         0  

[5 rows x 28 columns]

Need to create a file that's just one column for screen name, one column for retweet screenname, and one column for quoted screen name

In [27]:
df_names = df_double_verified.filter(['screen_name','retweet_screen_name','quoted_screen_name'], axis=1)

In [28]:
df_names.tail()

screen_name retweet_screen_name quoted_screen_name
1875   SubodhChandra                 NaN                AOC
2038  RealJamesWoods                 NaN                AOC
4066      bethfertig                 NaN        yasmeenkhan
374     RealSaavedra                 NaN                AOC
2208  MarkSKrikorian                 NaN                AOC

Creating graph object

AFTER DINNER:
 - need to merge retweet and quoted, create a column that just has one name
 - use the following code:
 
`
from_pandas_dataframe(df_names, screen_name, rt_quoted_screen_name, edge_attr=None, create_using=None)
`

In [29]:
df_names['rt_quoted_screen_name'] = df_names['retweet_screen_name']

In [30]:
df_names.head()

screen_name retweet_screen_name quoted_screen_name rt_quoted_screen_name
3494      aliklay             schwarz                NaN               schwarz
8     GreatDismal              ddale8                NaN                ddale8
38    GreatDismal              ddale8                NaN                ddale8
1015          YNB          VanityFair                NaN            VanityFair
1136         sxsw                sxsw                NaN                  sxsw

In [39]:
df_names['rt_quoted_screen_name'].fillna(df_names['quoted_screen_name'], inplace=True)


In [41]:
df_names.head()

screen_name retweet_screen_name quoted_screen_name rt_quoted_screen_name
3494      aliklay             schwarz                NaN               schwarz
8     GreatDismal              ddale8                NaN                ddale8
38    GreatDismal              ddale8                NaN                ddale8
1015          YNB          VanityFair                NaN            VanityFair
1136         sxsw                sxsw                NaN                  sxsw

~~hmm something's not right, come back to this after dinner~~
Now I want to create another column that indicated retweet or quote, so I can filter for this later

In [42]:
df_names['retweet_or_quote'] = np.where((df_names['quoted_screen_name'].isnull()), 'retweet', 'quote')

In [45]:
df_names.tail()

screen_name retweet_screen_name quoted_screen_name  \
1875   SubodhChandra                 NaN                AOC   
2038  RealJamesWoods                 NaN                AOC   
4066      bethfertig                 NaN        yasmeenkhan   
374     RealSaavedra                 NaN                AOC   
2208  MarkSKrikorian                 NaN                AOC   

     rt_quoted_screen_name retweet_or_quote  
1875                   AOC            quote  
2038                   AOC            quote  
4066           yasmeenkhan            quote  
374                    AOC            quote  
2208                   AOC            quote

In [50]:
G = nx.from_pandas_edgelist(df_names, source='screen_name', target='rt_quoted_screen_name', edge_attr='retweet_or_quote', create_using=None)



OK I may need to groupby source and target, and count # of retweets and quotes separately, and have those both be weights

In [51]:
print(nx.info(G))

Name: 
Type: Graph
Number of nodes: 2316
Number of edges: 3090
Average degree:   2.6684


In [52]:
nx.write_gexf(G, "verified.gexf")

using Louvain Modularity to find communities in my data

In [61]:
import community

In [62]:
partition = community.best_partition(G)

In [65]:
partition

{'aliklay': 0,
 'schwarz': 0,
 'GreatDismal': 1,
 'ddale8': 1,
 'YNB': 2,
 'VanityFair': 2,
 'sxsw': 3,
 'latinovictoryus': 4,
 'ErikaOSanoja': 5,
 'RaniaKhalek': 5,
 'michi83': 6,
 'eScarry': 6,
 'itsYourGrace': 7,
 'RealSaavedra': 7,
 'holamellamoboza': 4,
 'slowbeard': 0,
 'timothypmurphy': 0,
 'alexkotch': 0,
 'k_Johansen': 4,
 'jihadwatchRS': 7,
 'redsteeze': 1,
 '2Hats1Mike': 8,
 'fabiochiusi': 8,
 'Chet_Cannon': 1,
 'pnjaban': 9,
 'Harryslaststand': 4,
 'tmckenna1': 4,
 'DesJarlaisTN04': 7,
 'GovMikeHuckabee': 7,
 'guypbenson': 1,
 'ChadPergram': 1,
 'ChrisMcPhersn': 10,
 'decider': 10,
 'daweiner': 8,
 'AdamWeinstein': 8,
 'tstevensradio': 7,
 'WaltonNJohnson': 7,
 'MatthewTyrmand': 7,
 'dbongino': 7,
 'SebGorka': 7,
 'tonykchoi': 11,
 'VoteRunLead': 11,
 'andrewperezdc': 11,
 'GregSpeedDC': 0,
 'KatrinaPierson': 7,
 'DailyCaller': 1,
 'RashidaTlaib': 11,
 'OmanReagan': 12,
 'AlexCKaufman': 12,
 'longdrivesouth': 12,
 'gaywonk': 12,
 'MaxBlumenthal': 5,
 'BenjaminNorton': 5,
 '

found 167 communities

In [66]:
max(partition.values())

167